In [ ]:
# ================================================
# NETTOYAGE, FUSION ET PRÉPARATION DES DONNÉES
# Projet: Spéculation immobilière vs Accès au logement - Paris
# ================================================

print("PHASE DE NETTOYAGE ET PRÉPARATION DES DONNÉES")
print("=" * 70)

# ----------------------------------------------------
# 1. CONFIGURATION ET CHARGEMENT
# ----------------------------------------------------
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Chemins
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"
EXT_DIR = DATA_DIR / "external"

print("Chargement des données...")

# Charger les données DVF
df_dvf = pd.read_csv(RAW_DIR / "dvf_paris_intramuros.csv", low_memory=False)
print(f"✓ DVF chargées: {len(df_dvf):,} transactions")

# Charger les données INSEE
df_insee = pd.read_csv(RAW_DIR / "insee_arrondissements_paris.csv", low_memory=False)
print(f"✓ INSEE chargé: {len(df_insee)} arrondissements")

# Charger les contours géographiques
gdf_geo = gpd.read_file(EXT_DIR / "arrondissements_paris.geojson")
print(f"✓ Contours chargés: {len(gdf_geo)} polygones")

# ----------------------------------------------------
# 2. NETTOYAGE DES DONNÉES DVF
# ----------------------------------------------------
print("\n" + "=" * 70)
print("2. NETTOYAGE DES DONNÉES DVF")
print("=" * 70)

# 2.1. Suppression des colonnes inutiles (100% valeurs manquantes) (basé sur des analyses de valeurs manquantes au brouillon)
print("\n2.1. Suppression des colonnes inutiles...")
colonnes_inutiles = [
    'ancien_code_commune', 'ancien_nom_commune', 'ancien_id_parcelle',
    'code_nature_culture_speciale', 'nature_culture_speciale'
]
colonnes_existantes = [col for col in colonnes_inutiles if col in df_dvf.columns]
df_dvf_clean = df_dvf.drop(columns=colonnes_existantes)
print(f"   {len(colonnes_existantes)} colonnes supprimées")

# 2.2. Filtrage des transactions pertinentes
print("\n2.2. Filtrage des transactions...")
# Garder seulement les ventes (pas les échanges, adjudications, etc.)
if 'nature_mutation' in df_dvf_clean.columns:
    df_dvf_clean = df_dvf_clean[df_dvf_clean['nature_mutation'] == 'Vente']
    print(f"   ✓ Seules les ventes conservées")

# Garder seulement les logements (maisons et appartements)
if 'type_local' in df_dvf_clean.columns:
    mask_logement = df_dvf_clean['type_local'].isin(['Maison', 'Appartement'])
    df_dvf_clean = df_dvf_clean[mask_logement]
    print(f"   ✓ Seules les maisons/appartements conservés")

# 2.3. Nettoyage des valeurs aberrantes
print("\n2.3. Nettoyage des valeurs aberrantes...")

# Prix au m² aberrant (extrêmes)
if 'prix_m2' in df_dvf_clean.columns:
    # Supprimer les valeurs négatives ou nulles
    df_dvf_clean = df_dvf_clean[df_dvf_clean['prix_m2'] > 0]
    
    # Calculer les bornes pour éliminer les extrêmes (1er et 99e percentiles)
    Q1 = df_dvf_clean['prix_m2'].quantile(0.01)
    Q99 = df_dvf_clean['prix_m2'].quantile(0.99)
    df_dvf_clean = df_dvf_clean[(df_dvf_clean['prix_m2'] >= Q1) & (df_dvf_clean['prix_m2'] <= Q99)]
    print(f"   ✓ Prix/m²: {len(df_dvf_clean):,} transactions après nettoyage")
    print(f"     Plage retenue: {Q1:.0f} à {Q99:.0f} €/m²")

# Surface aberrante
if 'surface_reelle_bati' in df_dvf_clean.columns:
    df_dvf_clean = df_dvf_clean[(df_dvf_clean['surface_reelle_bati'] >= 9) & 
                                 (df_dvf_clean['surface_reelle_bati'] <= 500)]
    print(f"   ✓ Surface: 9 à 500 m² conservés")

# Valeur foncière aberrante
if 'valeur_fonciere' in df_dvf_clean.columns:
    df_dvf_clean = df_dvf_clean[(df_dvf_clean['valeur_fonciere'] >= 10000) & 
                                 (df_dvf_clean['valeur_fonciere'] <= 5000000)]
    print(f"   ✓ Valeur: 10k à 5M € conservés")

# 2.4. Calcul d'indicateurs DVF par arrondissement
# Nombre de transactions, Prix moyen au m², Prix médian au m², Écart-type des prix, 3ème quartile (Q3) des prix au m², valeur foncière médiane, surface réelle médiane.

print("\n2.4. Calcul des indicateurs par arrondissement...")

if 'arrondissement' in df_dvf_clean.columns and 'prix_m2' in df_dvf_clean.columns:
    dvf_par_arr = df_dvf_clean.groupby('arrondissement').agg({
        'prix_m2': ['count', 'mean', 'median', 'std', lambda x: x.quantile(0.75)],
        'valeur_fonciere': 'median',
        'surface_reelle_bati': 'median'
    }).round(2)

    # Renommer les colonnes
    dvf_par_arr.columns = [
        'nb_transactions', 'prix_m2_mean', 'prix_m2_median', 
        'prix_m2_std', 'prix_m2_q3', 'valeur_median', 'surface_median'
    ]
    
    dvf_par_arr = dvf_par_arr.reset_index()
    print(f"   ✓ {len(dvf_par_arr)} arrondissements agrégés")
    
    # Ajouter un indicateur de spéculation : le rapport entre le 3ème quartile (Q3) et le 2ème quartile (médianne) des prix au m²
    # Un grand rapport indique qu'une partie du marché s'envolle (marché à deux vitesses), i.e une pression spéculative à la hausse
    dvf_par_arr['spec_ratio'] = dvf_par_arr['prix_m2_q3'] / dvf_par_arr['prix_m2_median']
    print(f"   ✓ Indicateur de spéculation calculé (ratio Q3/médian)")
else:
    print("!!! Impossible de calculer les indicateurs DVF")
    dvf_par_arr = None

# ----------------------------------------------------
# 3. NETTOYAGE DES DONNÉES INSEE
# ----------------------------------------------------
print("\n" + "=" * 70)
print("3. NETTOYAGE DES DONNÉES INSEE")
print("=" * 70)

# 3.1. Suppression des colonnes inutiles
print("\n3.1. Suppression des colonnes inutiles...")
# Colonnes avec 100% de valeurs manquantes (basé sur des analyses de valeurs manquantes au brouillon)
colonnes_100pct_manquantes = [col for col in df_insee.columns 
                              if col.startswith(('P22_RP_ELEC', 'P22_RP_EAUCH', 'P22_RP_BDWC'))]
colonnes_existantes = [col for col in colonnes_100pct_manquantes if col in df_insee.columns]
df_insee_clean = df_insee.drop(columns=colonnes_existantes)
print(f"   {len(colonnes_existantes)} colonnes avec 100% de NaN supprimées")

# 3.2. Sélection des variables pertinentes pour l'analyse
print("\n3.2. Sélection des variables pertinentes...")

# Variables de LOGEMENT (spéculation et accessibilité)
variables_logement = [
    'P22_LOG',          # Nombre total de logements
    'P22_LOGVAC',       # Logements vacants (SPÉCULATION)
    'P22_MAISON',       # Maisons
    'P22_APPART',       # Appartements
    'P22_RPMAISON',     # Résidences principales maisons
    'P22_RPAPPART',     # Résidences principales appartements
]

# Variables de SPÉCULATION (historique des logements vacants)
variables_speculation = [
    'P16_LOGVAC', 'P11_LOGVAC', 'P06_LOGVAC',  # Évolution des logements vacants
    'D99_LOGVAC', 'D90_LOGVAC', 'D82_LOGVAC',  # Données détaillées historiques
    'D75_LOGVAC', 'D68_LOGVAC'
]

# Variables DÉMOGRAPHIQUES (proxy pour accessibilité)
variables_demo = [
    'P22_POP',          # Population
    'C22_MEN',          # Nombre de ménages
    'C22_MENPSEUL',     # Ménages d'une personne
]

# Combiner toutes les variables pertinentes
variables_selectionnees = ['arrondissement'] + \
                         [v for v in variables_logement if v in df_insee_clean.columns] + \
                         [v for v in variables_speculation if v in df_insee_clean.columns] + \
                         [v for v in variables_demo if v in df_insee_clean.columns]

# Garder seulement les colonnes sélectionnées
df_insee_selection = df_insee_clean[variables_selectionnees].copy()
print(f"   ✓ {len(variables_selectionnees)} variables conservées sur {len(df_insee.columns)}")

# 3.3. Calcul d'indicateurs composites
print("\n3.3. Calcul d'indicateurs composites...")

# Taux de vacance = (logements vacants / total logements) × 100
# (un autre indicateur de spéculation ou de mauvaise allocation des ressources par le marché)
if 'P22_LOG' in df_insee_selection.columns and 'P22_LOGVAC' in df_insee_selection.columns:
    df_insee_selection['taux_vacance'] = (df_insee_selection['P22_LOGVAC'] / df_insee_selection['P22_LOG']) * 100
    print(f"   ✓ Taux de vacance calculé (%)")

# Part des ménages seuls = (ménages d'une seule personne / total ménages) × 100
# (un proxy pour la précarité)
if 'C22_MEN' in df_insee_selection.columns and 'C22_MENPSEUL' in df_insee_selection.columns:
    df_insee_selection['part_menages_seuls'] = (df_insee_selection['C22_MENPSEUL'] / df_insee_selection['C22_MEN']) * 100
    print(f"   ✓ Part des ménages seuls calculée (%)")

# ----------------------------------------------------
# 4. FUSION DES DONNÉES DVF ET INSEE
# ----------------------------------------------------
print("\n" + "=" * 70)
print("4. FUSION DES DONNÉES DVF ET INSEE")
print("=" * 70)

if dvf_par_arr is not None and 'arrondissement' in df_insee_selection.columns:
    # Fusion sur l'arrondissement
    df_fusion = pd.merge(dvf_par_arr, df_insee_selection, on='arrondissement', how='inner')
    
    print(f"✓ Fusion réussie: {len(df_fusion)} arrondissements")
    print(f"  Colonnes totales: {len(df_fusion.columns)}")
    
    # Calcul d'indicateurs finaux
    print("\n4.1. Calcul des indicateurs finaux...")
    
    # Ratio prix/m² moyen par taux de vacance (i.e Combien coûte 1% de vacance en prix moyen du m²)
    # (Je ne l'utiliserai pas)
    if 'prix_m2_mean' in df_fusion.columns and 'taux_vacance' in df_fusion.columns:
        df_fusion['ratio_prix_vacance'] = df_fusion['prix_m2_mean'] / df_fusion['taux_vacance']
        print(f"   ✓ Ratio prix/vacance calculé")
    
    # Indicateur de pression spéculative composite
    if all(col in df_fusion.columns for col in ['spec_ratio', 'taux_vacance', 'prix_m2_mean']):
        # Normalisation des variables (moyenne=0, variance=1) pour qu'elles aient la même importance dans l'indicateur
        df_fusion['spec_norm'] = (df_fusion['spec_ratio'] - df_fusion['spec_ratio'].mean()) / df_fusion['spec_ratio'].std()
        df_fusion['vac_norm'] = (df_fusion['taux_vacance'] - df_fusion['taux_vacance'].mean()) / df_fusion['taux_vacance'].std()
        df_fusion['prix_norm'] = (df_fusion['prix_m2_mean'] - df_fusion['prix_m2_mean'].mean()) / df_fusion['prix_m2_mean'].std()
        
        # Indice composite (moyenne arithmétique des 3)
        # Notre indice composite capture la spéculation immobilière dans sa triple dimension : l'inégalité interne du marché (écart Q3/médian), la vacance anormale (logements vides) et le niveau absolu des prix.
        # Contrairement aux indicateurs unidimensionnels comme le simple ratio prix/vacance qui ne mesure que la tension du marché,
        # notre indice identifie les comportements véritablement spéculatifs où se combinent rétention de logements (offre potentielle élevée), fortes disparités de prix et cherté du marché.
        # Cette approche explique pourquoi certains arrondissements chers mais homogènes (comme le 6ème) ont un indice modéré, tandis que d'autres révèlent une spéculation active malgré des prix relativement moindres.
        df_fusion['indice_speculation'] = (df_fusion['spec_norm'] + df_fusion['vac_norm'] + df_fusion['prix_norm']) / 3
        print(f"   ✓ Indice composite de spéculation calculé")
    
    
    # Sauvegarde
    output_path = PROC_DIR / "donnees_fusionnees_paris.csv"
    df_fusion.to_csv(output_path, index=False)
    print(f"\n✓ Données fusionnées sauvegardées: {output_path}")
    
else:
    print("!!! Impossible de fusionner: données DVF ou INSEE incomplètes")
    df_fusion = None

# ----------------------------------------------------
# 5. PRÉPARATION DES DONNÉES GÉOGRAPHIQUES
# ----------------------------------------------------
print("\n" + "=" * 70)
print("5. PRÉPARATION DES DONNÉES GÉOGRAPHIQUES")
print("=" * 70)

# Standardisation de la colonne d'arrondissement dans gdf_geo (i.e mettre au même format que dans les deux autres bases)
if 'l_ar' in gdf_geo.columns:
    gdf_geo['arrondissement'] = gdf_geo['l_ar'].str.extract(r'(\d+)').astype(int)
    print(f"✓ Colonne 'arrondissement' ajoutée aux contours")

# Fusion avec les données statistiques (avec la fusion précédente DVF et INSEE)
if df_fusion is not None and 'arrondissement' in gdf_geo.columns:
    gdf_complet = gdf_geo.merge(df_fusion, on='arrondissement', how='left')
    
    # Sauvegarde
    geo_output_path = PROC_DIR / "paris_arrondissements_complet.geojson"
    gdf_complet.to_file(geo_output_path, driver='GeoJSON')
    print(f"✓ Données géographiques enrichies sauvegardées: {geo_output_path}")
else:
    print("!!!  Données géographiques non fusionnées avec les statistiques")

# ----------------------------------------------------
# 6. SYNTHÈSE ET VÉRIFICATION FINALE
# ----------------------------------------------------
print("\n" + "=" * 70)
print("6. SYNTHÈSE FINALE")
print("=" * 70)

if df_fusion is not None:
    print(f"\nDONNÉES FUSIONNÉES DISPONIBLES:")
    print(f"   • Arrondissements: {len(df_fusion)}")
    print(f"   • Variables: {len(df_fusion.columns)}")
    
    print(f"\nINDICATEURS CLÉS CALCULÉS:")
    indicateurs = ['nb_transactions', 'prix_m2_median', 'taux_vacance','spec_ratio',
                   'indice_speculation']
    indicateurs_existants = [i for i in indicateurs if i in df_fusion.columns]
    
    for ind in indicateurs_existants:
        print(f"   • {ind}: {df_fusion[ind].mean():.2f} (moyenne)")
    
    print(f"\n TOP 5 DES ARRONDISSEMENTS LES PLUS SPÉCULATIFS:")
    if 'indice_speculation' in df_fusion.columns:
        top_spec = df_fusion.nlargest(5, 'indice_speculation')[['arrondissement', 'indice_speculation', 'prix_m2_median',]]
        for _, row in top_spec.iterrows():
            print(f"   Arr. {row['arrondissement']}: indice={row['indice_speculation']:.2f}, prix/m²={row['prix_m2_median']:.0f}€")
    
    print(f"\n CORRÉLATIONS CLÉS (à explorer):")
    if all(col in df_fusion.columns for col in ['prix_m2_median', 'taux_vacance', 'spec_ratio']):
        corr_vacance = df_fusion['prix_m2_median'].corr(df_fusion['taux_vacance'])
        corr_spec = df_fusion['prix_m2_median'].corr(df_fusion['spec_ratio'])
        print(f"   • Prix/m² médian vs Taux vacance: {corr_vacance:.3f}")
        print(f"   • Prix/m² médian vs Ratio spéculation: {corr_spec:.3f}")

print(f"\n FICHIERS CRÉÉS:")
print(f"   1. {PROC_DIR / 'donnees_fusionnees_paris.csv'} (données fusionnées)")
if 'gdf_complet' in locals():
    print(f"   2. {PROC_DIR / 'paris_arrondissements_complet.geojson'} (données géo enrichies)")

print(f"\n" + "=" * 70)
print("✓ NETTOYAGE ET FUSION TERMINÉS AVEC SUCCÈS")

PHASE DE NETTOYAGE ET PRÉPARATION DES DONNÉES
Chargement des données...
✓ DVF chargées: 80,964 transactions
✓ INSEE chargé: 20 arrondissements
✓ Contours chargés: 20 polygones

2. NETTOYAGE DES DONNÉES DVF

2.1. Suppression des colonnes inutiles...
   5 colonnes supprimées

2.2. Filtrage des transactions...
   ✓ Seules les ventes conservées
   ✓ Seules les maisons/appartements conservés

2.3. Nettoyage des valeurs aberrantes...
   ✓ Prix/m²: 34,137 transactions après nettoyage
     Plage retenue: 47 à 1058140 €/m²
   ✓ Surface: 9 à 500 m² conservés
   ✓ Valeur: 10k à 5M € conservés

2.4. Calcul des indicateurs par arrondissement...
   ✓ 20 arrondissements agrégés
   ✓ Indicateur de spéculation calculé (ratio Q3/médian)

3. NETTOYAGE DES DONNÉES INSEE

3.1. Suppression des colonnes inutiles...
   3 colonnes avec 100% de NaN supprimées

3.2. Sélection des variables pertinentes...
   ✓ 18 variables conservées sur 1977

3.3. Calcul d'indicateurs composites...
   ✓ Taux de vacance calculé (